In [1]:
import os
import numpy as np
import audiolabel
import pandas as pd
import parselmouth as pm

Start by trying everything out on a test recording. Make sure to change these filenames to whatever your names are!

In [2]:
testwav = './recordings/BJPwordlist2_smoothed.WAV'
testtg = './recordings/BJPwordlist2_smoothed.TextGrid'

audiolabel is a package Ron wrote to help us use textgrids!

In [3]:
tg = audiolabel.LabelManager(from_file=testtg, from_type='praat')
tg

LabelManager(tiers=[0,1], names=['Vowels,','Notes'])

We used to do more with audiolabel objects, but these days we just turn them into dataframes, because Pandas is so good for this kind of work. But notice (above) that the textgrid has two tiers, which we did on purpose to keep notes! But right now we'll ignore the notes because we want to make something that works. We can return to the notes later.

The function as_df() in audiolabel turns the audiolabel object into a DataFrame, but because there are two tiers, we will take just the top most tier which we'll assume has the vowels in it. We're also only interested in the intervals that have text in them, as those are where the vowels are!

What's in the DataFrame?

In [4]:
tgdf = tg.as_df()[0]
voweldf = tgdf[tgdf.text!='']
voweldf.head()

,t1,t2,text,duration,center
1,1.549226,1.706560,boit,0.157335,1.627893
3,3.478393,3.672667,bane,0.194274,3.575530
5,5.554749,5.708663,bean,0.153914,5.631706
7,7.552785,7.678653,boot,0.125868,7.615719
9,9.425289,9.574415,boon,0.149126,9.499852


Let's set up a few other columns in the DF. First, the times for each row that we would find 20%, 50%, and 80% through the vowel.

In [5]:
voweldf['t20'] = voweldf.apply(lambda x: x.t1+x.duration*.2, axis=1)
voweldf['t50'] = voweldf.apply(lambda x: x.t1+x.duration*.5, axis=1)
voweldf['t80'] = voweldf.apply(lambda x: x.t1+x.duration*.8, axis=1)
voweldf.head()

<ipython-input-5-f118ae36d082>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  voweldf['t20'] = voweldf.apply(lambda x: x.t1+x.duration*.2, axis=1)
<ipython-input-5-f118ae36d082>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  voweldf['t50'] = voweldf.apply(lambda x: x.t1+x.duration*.5, axis=1)
<ipython-input-5-f118ae36d082>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

,t1,t2,text,duration,center,t20,t50,t80
1,1.549226,1.706560,boit,0.157335,1.627893,1.580693,1.627893,1.675093
3,3.478393,3.672667,bane,0.194274,3.575530,3.517248,3.575530,3.633812
5,5.554749,5.708663,bean,0.153914,5.631706,5.585532,5.631706,5.677880
7,7.552785,7.678653,boot,0.125868,7.615719,7.577958,7.615719,7.653479
9,9.425289,9.574415,boon,0.149126,9.499852,9.455115,9.499852,9.544590


Let's change the `text` column to `word`, and we no longer need the `t1` and `t2` columns

In [6]:
voweldf = voweldf.rename(columns={'text':'word'}).drop(columns=['t1','t2'])
voweldf.head()

,word,duration,center,t20,t50,t80
1,boit,0.157335,1.627893,1.580693,1.627893,1.675093
3,bane,0.194274,3.575530,3.517248,3.575530,3.633812
5,bean,0.153914,5.631706,5.585532,5.631706,5.677880
7,boot,0.125868,7.615719,7.577958,7.615719,7.653479
9,boon,0.149126,9.499852,9.455115,9.499852,9.544590


Let's now grab the corresponding audio file! We can do this in a lot of ways but probably easiest to do it in `parselmouth`. This is basically a Praat extension for Python.

In [7]:
wav = pm.Sound(testwav)
[au, of, op, nf] = wav.extract_all_channels()

Now let's do something a little fancy. of, nf, and op are oral flow, nasal flow, and oral pressure Sound objects, and since we have the columns t20, t50, and t80, we can use the `apply` function in the pandas library to apply a function (in this case, the parselmouth Sound object function get a value at time x) to every row in the dataframe.

In [8]:
voweldf['oaf20'] = voweldf.t20.apply(lambda x: of.get_value(x))
voweldf['oaf50'] = voweldf.t50.apply(lambda x: of.get_value(x))
voweldf['oaf80'] = voweldf.t80.apply(lambda x: of.get_value(x))
voweldf.head()

,word,duration,center,t20,t50,t80,oaf20,oaf50,oaf80
1,boit,0.157335,1.627893,1.580693,1.627893,1.675093,0.126407,0.145078,0.153215
3,bane,0.194274,3.575530,3.517248,3.575530,3.633812,0.055660,0.090363,0.103088
5,bean,0.153914,5.631706,5.585532,5.631706,5.677880,0.111634,0.113190,0.109711
7,boot,0.125868,7.615719,7.577958,7.615719,7.653479,0.121855,0.112943,0.124084
9,boon,0.149126,9.499852,9.455115,9.499852,9.544590,0.104082,0.101866,0.144723


And now do the same for nasal flow and oral pressure!

In [9]:
voweldf['naf20'] = voweldf.t20.apply(lambda x: nf.get_value(x))
voweldf['naf50'] = voweldf.t50.apply(lambda x: nf.get_value(x))
voweldf['naf80'] = voweldf.t80.apply(lambda x: nf.get_value(x))
voweldf['oap20'] = voweldf.t20.apply(lambda x: op.get_value(x))
voweldf['oap50'] = voweldf.t50.apply(lambda x: op.get_value(x))
voweldf['oap80'] = voweldf.t80.apply(lambda x: op.get_value(x))
voweldf.head()

,word,duration,center,t20,t50,t80,oaf20,oaf50,oaf80,naf20,naf50,naf80,oap20,oap50,oap80
1,boit,0.157335,1.627893,1.580693,1.627893,1.675093,0.126407,0.145078,0.153215,0.029510,0.023377,0.022675,0.012200,0.010041,0.009815
3,bane,0.194274,3.575530,3.517248,3.575530,3.633812,0.055660,0.090363,0.103088,0.020264,0.038671,0.044952,0.006942,0.006547,0.005891
5,bean,0.153914,5.631706,5.585532,5.631706,5.677880,0.111634,0.113190,0.109711,0.014427,0.011460,0.021962,0.007425,0.005595,0.005188
7,boot,0.125868,7.615719,7.577958,7.615719,7.653479,0.121855,0.112943,0.124084,0.026581,0.026398,0.025299,0.008114,0.006612,0.007050
9,boon,0.149126,9.499852,9.455115,9.499852,9.544590,0.104082,0.101866,0.144723,0.024530,0.022950,0.026542,0.008540,0.006257,0.012318


And we might be interested in saving that as an external file.

In [12]:
voweldf.to_csv('./test.csv')